# ECRH parameterisation comparison

First, we look at what is currently used in STEP:

In [1]:
import jetto_tools
import plotly.graph_objects as go
import plotly
import numpy as np


plotly.io.templates.default = "simple_white"

template = jetto_tools.template.from_directory("../jetto/templates/spr45-v9")
exfile = jetto_tools.binary.read_binary_file(template.extra_files["jetto.ex"])

# Normalise ECRH to [0, 1]
benchmark_ecrh = exfile["QECE"][0] / np.max(exfile["QECE"][0])
xrho = exfile["XRHO"][0]

figure = go.Figure()
figure.add_trace(go.Scatter(x=xrho, y=benchmark_ecrh, name="SPR45-v9"))
figure.update_xaxes(title="Normalised radius")
figure.update_yaxes(title="Normalised ECRH power density")

15:51:45 [WARNING] Python module 'omas' not found. Submodule 'jams' needs it @jams.py:14


Load our ECRH functions:

In [2]:
from jetto_mobo.ecrh import (
    piecewise_linear,
    piecewise_linear_decreasing,
    sum_of_gaussians,
)
from jetto_mobo.genetic_algorithm import piecewise_linear as ga_piecewise_linear

f_dict = {
    ga_piecewise_linear: 12,
    piecewise_linear: 12,
    piecewise_linear_decreasing: 12,
    sum_of_gaussians: 12,
}

Fit to benchmark:

In [3]:
from scipy.optimize import curve_fit

parameters = {}
for f, n_params in f_dict.items():
    optimised_parameters, _ = curve_fit(
        f=lambda x, *p: f(x, p),
        xdata=xrho,
        ydata=benchmark_ecrh,
        p0=[
            *np.linspace(0.2, 0.5, n_params // 2),
            *np.linspace(1, 0.1, n_params // 2),
        ],
        bounds=(0, 1),
        max_nfev=1e4,
    )
    parameters[f.__name__] = optimised_parameters

    figure.add_trace(
        go.Scatter(
            x=xrho,
            y=f(xrho, optimised_parameters),
            name=f.__name__,
        )
    )

figure.show()

Example random parameterisations:

In [4]:
for f, n_params in f_dict.items():
    figure = go.Figure()
    for i in range(5):
        figure.add_trace(
            go.Scatter(
                x=xrho,
                y=f(xrho, np.random.rand(n_params)),
                name=str(i)
            )
        )
    figure.update_layout(
        title=f.__name__,
        xaxis_title="Normalised radius",
        yaxis_title="Normalised ECRH power density"
    )
    figure.show()